# Microsoft Movie Studio Project

**Author**: Jonny Hofmeister

## Overview

This project explores movie data from IMDB and Box Office Mojo to generate business insights for a new movie studio.

## Business Problem

Microsoft has decided to enter the movie production industry and create a new movie studio, but they want some advice. This project asks and analyzes which types of films are doing best at the box office, and the results will inform decisions and actions Microsoft's studio can take to create successful production company and films.

<img src="./images/studio.jpeg" width="700"/>

## Data Understanding

Box Office Mojo (BOM) is a webside run by IMDB that reports and organizes box office receipts from Movies. The essential data being pulled from this source is a Domestic and Foreign Gross for correspoding movies identified by a title. 

IMDB is a webside containing extensive details about individual movies. The datasets contain information like titles, year released, runtime, production, persons involed (actors, diretors, etc), and importantly, review data associated with each movie. 

We are given access to more sets of data, but for simplicity and consistency, this analysis will focus on a dataset created from the combination of these two sources.

### Import necessary packages

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mp
import seaborn as sns

from fuzzywuzzy import process, fuzz

In [2]:
# Filtering warnings to exclude unwanted output

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Import Data

In [3]:
# Load .CSV data using into DataFrames using Pandas

bom = pd.read_csv('data/bom.movie_gross.csv')
imdb_basics = pd.read_csv('data/imdb.title.basics.csv')
imdb_ratings = pd.read_csv('data/imdb.title.ratings.csv')

In [4]:
# First check out how the IMDB_basics data is organized

imdb_basics.head(2)

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"


In [5]:
imdb_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [6]:
# Now take a peek at the IMDB_ratings dataset

imdb_ratings.head(2)

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559


In [7]:
imdb_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


### Data exploration

The IMDB Basics table includes essential information like movie titles and the genres they are associated with. The genres are stored in a single string, and each movie contains up to 3 associated genres. The data set contains 146,000 movies, quite a significant amount.

The IMDB Ratings table contains information on the movies average rating and the number of rating votes cast for that movie. The number of review movies is about 74000, about half that of the basics datatable, but still a very large amount, more than enough to work with. Movies that don't have ratings are also likely to be smaller, less-known or viewed and therefore less interesting to us from an analysis perspective as well.

We can see that both datasets include a column called 'tconst' which appears to be a unique identifier for each movie. This will work perfect to join both data tables. First lets check that 'tconst' is indeed unique and contains no duplicates.

In [8]:
imdb_basics.tconst.duplicated().value_counts()

False    146144
Name: tconst, dtype: int64

Perfect, 'tconst' is a unique identifier that contains no duplicates. This will work as the perfect index to join these two dataframes when the time comes.

Next let's get aquianted with the Box Office Mojo (BOM) data.

In [9]:
bom.head(2)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010


In [10]:
bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


BOM contains the very important gross information we are interested in for each movie title, as well as the year released, which will come in handy. The dataset is about 3300 movies, much smaller than the IMDB data. While we can't work with a gigantic set of movies anymore, a few thousand of the most important films still should be enough to glean some information on which movies have been most sucessful. I am going to explore success of movies though gross and also through rating, but when giving advice to Microsoft in building their studio, the more important information is the gross and how much money the movie made. 

## Data Preparation

We have a few items on the list to get the data ready to extract the information we want into figures. This includes cleaning the 'genres' column into something more useful than a string; merging the two IMDB dataframes on the 'tconst' index; and joining those to the BOM dataset by matching titles.

Matching titles could be done through cleaning methods like, making everything lowercase, removing punctuation, adding a year to the title, and then joining to hopefully get more correct matches. Instead of this, I have opted to utilize the FuzzyWuzzy package which applies the Levenshtein distance formula among other techniques to match string, returning the best match and a value rating how similar Fuzzy thinks the two strings are (value between 0 and 100). 

### Fuzzy matching

The script I have created to match BOM movie titles with IMDB movie titles is included in a separate .ipynb file for certain reasons. The main one being that the advanced searching algorithm took about an hour to run. For the sake of running this analysis notebook in its entirety, I have done this separately. 

Please refer to the bom_fuzzy.ipynb file for details on how this was done. At the end of the fuzzy script, I have saved the BOM dataframe, which includes matched titles, to a new .CSV file which I will import below and continue cleaning and merging. As the /data folder is ignored by my git repo, this .csv file in located in the home folder of the phase1_project repository.

In [11]:
# Let's take a peek at what these matches look like

bom_fuzzy_matches = pd.read_csv('bom_fuzzy_matches.csv')

In [12]:
bom_fuzzy_matches.head()

,title,studio,domestic_gross,foreign_gross,year,matched_title,similarity
0,Toy Story 3 2010,BV,415000000.0,652000000,2010,Toy Story 3 2010,100
1,Alice in Wonderland (2010) 2010,BV,334200000.0,691300000,2010,Alice in Wonderland 2010,87
2,Harry Potter and the Deathly Hallows Part 1 2010,WB,296000000.0,664300000,2010,Harry Potter and the Deathly Hallows: Part 1 2010,99
3,Inception 2010,WB,292600000.0,535700000,2010,Inception 2010,100
4,Shrek Forever After 2010,P/DW,238700000.0,513900000,2010,Shrek Forever After 2010,100


In [13]:
bom_fuzzy_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
 5   matched_title   3387 non-null   object 
 6   similarity      3387 non-null   int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 185.4+ KB


Here in the matched dataframe, we still have the 'title' column which is the same as the titles in the original BOM data, the corresponding gross data and year. The two last columns are new. 'matched_title' is the title from the IMDB dataset that Fuzzy found to be most similar, as well as the similarity rating returned by the matching algorithm. We can filter the data by using this 'similarity' to include only matches we know Fuzzy is certain are the same. 

It is also important to note that the title and matched_title columns were updated before the matching process to include the year released after the title. This helped improve matching success. I would leave the years in there, as we wont be plotting by movie title, but in order to join this DataFrame with imdb_basics, I will have to remove the year in order to match the titles properly.

Another important note: As this was completely a few days ago, BOM and imdb_basics was fuzzy matched before I did any manipulation to the IMDB datasets. In the future I would do this in the opposite order. For example, first joining the imdb_basics and imdb_ratings would cut the size of our data in half, which in turn would reduce the Fuzzy matching runtime by about half. Given this still takes a while to re-run, and I have already saved the new .csv matching file, I will just continue to clean the data from here. We will end up with the same data in the end, but be aware this could have been done in a different order to improve efficiency. 

#### Before joining BOM and IMDB.....
I will join and clean the imdb_basics and imdb_ratings dataset, including cleaning the genres column to create a list of genres and a column for each with a True/False binary value to determine if the movie is in that genre. This will make selecting movies where a specific genre == True very simple.

I am also doing this before the BOM megre because that merge will decrease our dataset to only 3000 rows. I may also be interested in exploring the ratings of all IMDB moives without the BOM gross data. So first create a merged set of Basics and Ratings that inlcudes rating and genre information for 70k movies. Then this Dataframe is what I will join with the BOM fuzzy matches. This allows for slightly more flexibility in the size of the dataset while doing analysis and exploration. 

In [14]:
# First identify the index column for the merge, in this case it is the 'tconst' ID.

imdb_basics.set_index('tconst', inplace=True)
imdb_ratings.set_index('tconst', inplace=True)

In [15]:
# Now join. This is an inner join because we only want rows that have IDs in both dataframes in our new DF

basics_ratings = imdb_basics.join(imdb_ratings, how='inner')
basics_ratings.head()

,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
tconst,,,,,,,
tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43
tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517
tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13
tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119


In [16]:
len(basics_ratings)

73856

We can see here this was a very sucessful join as the length of the result is the same as the length of the imdb_ratings dataframe. This means that each 'tconst' was indeed unique and every single movie in the ratings DF was found in the Basics DF.

In [17]:
# To eliminate confusion, I am going to drop the 'original_title' column from this DF. Primary_title was
# the one used in the fuzzy matching. So to not get mixed up, lets just delete the other title column.

# Note: original_title is generally the same as primary, but in some cases is a title in a different language,
# and primary title matched better witht the title in the BOM data

basics_ratings.drop(columns = 'original_title', inplace=True)

# Check it worked
basics_ratings.head(2)

,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes
tconst,,,,,,
tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77
tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43


The next step is to manipulate the genres column into something useful.
The steps I will go through to do this are first clean the genres string and split it by the commas, appending the individual genres into a list object containing anywhere from 0 to 3 genres. Then I will search through all genres to create a list of all genre types, then use this list to create a new column for each genre and ask each row if that genre is in the genres_list or not, and create a boolean value in that genre column if it is or isn't true.

In [18]:
# Here we split the genres column by the ',' delimiter and append to a new column containing individual
# genres in list format.

basics_ratings['list_genres'] = basics_ratings['genres'].map(lambda x: str(x).split(','))

basics_ratings.head(2)

,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes,list_genres
tconst,,,,,,,
tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,"[Action, Crime, Drama]"
tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43,"[Biography, Drama]"


Now let's make a list of all the genre types:

In [19]:
list_of_genre_types = []
for i in range(len(basics_ratings)):
    for x in basics_ratings['list_genres'][i]:
        if x in list_of_genre_types:
            continue
        elif x not in list_of_genre_types:
            list_of_genre_types.append(x)

A bit of genre cleaning: sorting genres alphabetically will be useful for bar plots.
Also, the 'Short' films genre only contains one entry, so it is basically useless.
The NaN genre is also not of any interest to us, so let's remove those genres right now too.

In [20]:
list_of_genre_types.sort()

list_of_genre_types.remove('Short')
list_of_genre_types.remove('nan')
list_of_genre_types

['Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Sport',
 'Thriller',
 'War',
 'Western']

Sweet, we have all the genres organized in a useful list now. Next step is to iterate through this list, creating a new column in our dataframe for each genre that contains a True or False if that genre is contained in the movie's 'list_genres' list.

In [21]:
for genre in list_of_genre_types:
    basics_ratings[genre] = basics_ratings['list_genres'].map(lambda x: True if (genre in x) else False)

In [22]:
basics_ratings.head(3)

,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes,list_genres,Action,Adult,Adventure,...,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Thriller,War,Western
tconst,,,,,,,,,,,,,,,,,,,,,
tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,"[Action, Crime, Drama]",True,False,False,...,False,False,False,False,False,False,False,False,False,False
tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43,"[Biography, Drama]",False,False,False,...,False,False,False,False,False,False,False,False,False,False
tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,[Drama],False,False,False,...,False,False,False,False,False,False,False,False,False,False


Perfect, now this dataframe and genres column can be selected to pull only movies that are in a certain genre.
This would look like "basics_ratings[basics_ratings['Action'] == True]' to choose only Action movies.

### Joining BOM data

Now that the IMDB data is prepped, we can continue to merge in the BOM gross data.

First, remove the year from the 'matched_title' in the bom_fuzzy_matches dataframe. Then merge that dataset with the imdb dataset on the title columns. 

In [23]:
# Can easily remove the year just by removing the last 5 characters from the title string.

bom_fuzzy_matches['matched_title'] = bom_fuzzy_matches['matched_title'].map(lambda x: str(x)[:-5])

In [24]:
# Check it worked

bom_fuzzy_matches['matched_title'].head(2)

0            Toy Story 3
1    Alice in Wonderland
Name: matched_title, dtype: object

One more thing before merging: lets only select the movies in the bom_fuzzy_matches dataframe where the FuzzyWuzzy was very sure the titles matched. I decided on the similarity cutoff value by scrolling through the dataframe with similarity < 100 and examining which ones were correct or not.

While similarties in the 80s were occasionally wrong, anything I found above 90 during my brief scroll had been matched correctly. We could lower this value a bit further, but the number of rows above 90 is 2789 out of the initial 3387. I am happy continuing with 82% of our data, this is a significant enough fraction. It is a benefit to know we are continuing with only matched data we are certain about than include mis-matches.

In [25]:
bom_matches = bom_fuzzy_matches[bom_fuzzy_matches['similarity'] > 90]

In [26]:
bom_matches.head()

,title,studio,domestic_gross,foreign_gross,year,matched_title,similarity
0,Toy Story 3 2010,BV,415000000.0,652000000,2010,Toy Story 3,100
2,Harry Potter and the Deathly Hallows Part 1 2010,WB,296000000.0,664300000,2010,Harry Potter and the Deathly Hallows: Part 1,99
3,Inception 2010,WB,292600000.0,535700000,2010,Inception,100
4,Shrek Forever After 2010,P/DW,238700000.0,513900000,2010,Shrek Forever After,100
5,The Twilight Saga: Eclipse 2010,Sum.,300500000.0,398000000,2010,The Twilight Saga: Eclipse,100


Also before we do the final merge, lets check for movie duplicates and remove them from each dataframe.

In [27]:
bom_matches.duplicated(subset='matched_title').value_counts()

False    2780
True        9
dtype: int64

In [28]:
basics_ratings.duplicated(subset='primary_title').value_counts()

False    69993
True      3863
dtype: int64

In [30]:
bom_matches = bom_matches.drop_duplicates(subset='matched_title')
basics_ratings = basics_ratings.drop_duplicates(subset='primary_title')

Now we have removed all the duplicate titles. This will ensure that we dont create extra duplicate rows while we do the final join. 

Some of these duplicate titles are becuase a movie was re-made or re-released in a different year. Unfortinately, our initial data in either dataframe is not specific enough to tell which may be which, and does often contain the same release year. To make it easy and quick, I have decided to get rid of all duplicates and keep the first. Most of these are real duplicates but most likely we are also losing a few movies that were released multiple times and have different gross and rating data for each. If the initial data was more clear it might be easy to create separate titles or IDs for these, but given my time constraint, I have elected to drop them rather than accidentally include duplicates.

Now we merge bom_matches with basics_ratings using an inner merge to select where matched_title and primary_title are the same.

In [32]:
df = bom_matches.merge(basics_ratings, left_on='matched_title', right_on='primary_title', how='inner')

Great! This new dataframe now includes the movies the are included in both the BOM and IMDB dataset, so we can examine both gross and ratings and genre information for all of them. 

Examing the length of the result to the original, we only lost 13 rows from bom_matches. 2767 is the final length of our movie dataframe. Not terrible considering we started with 3387.

## Plotting